# RFP: Maze Solvers

## Project Overview
You are invited to submit a proposal that answers the following question:

### What path will your elf take?

*Please submit your proposal by **2/11/25 at 11:59 PM**.*

## Required Proposal Components

### 1. Data Description
In the code cell below, use [Gymnasium](https://gymnasium.farama.org/) to set up a [Frozen Lake maze](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) for your project. When you are done with the set up, describe the reward system you plan on using.

*Note, a level 5 maze is at least 10 x 10 cells large and contains at least five lake cells.*

In [1]:
import pandas as pd
import gymnasium as gym
from gymnasium.envs.toy_text.frozen_lake import generate_random_map
import random

In [2]:
msize=10
seedd=69

In [92]:
# Make maze
env = gym.make('FrozenLake-v1',desc=generate_random_map(size=msize,seed=seedd),render_mode="human")
initial_state = env.reset()
env.render()


#env.close()

In [94]:
env.close()

#### so im thinking of maybe like -1 for each space -1 for each move than like -400 for hole and like  +200 for gift so if its trying to get the best score it will find the least amount of moves 

### 2. Training Your Model
In the cell seen below, write the code you need to train a Q-Learning model. Display your final Q-table once you are done training your model.

*Note, level 5 work uses only the standard Python library and Pandas to train your Q-Learning model. A level 4 uses external libraries like Baseline3.*

In [5]:
# Train model here.
# Don't forget to display your final Q table!

In [77]:
alpha = 0.1
gam = 0.9 
eps = 0.1 
numep = 1000
nact=env.action_space.n
ns=msize * msize
stat=0
goalstate = ns - 1

In [39]:
Qt = {state: [0] * nact for state in range(ns)}

In [87]:
df = pd.DataFrame(Qt)

In [88]:
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,2.592246,1.696122,0,-0.764200,0.286461,0,-1.0,-0.1,0.1,0,...,-1.039196,0,-6.942502,0,-0.570216,-0.482604,3.351246,0,0,0
1,2.678051,0.132106,0,-0.000681,0.000000,0,0.0,0.0,0.0,0,...,-3.075347,0,-7.090705,0,-2.344660,0.193561,-2.797479,0,0,0
2,2.770777,-2.677126,0,0.075009,0.000000,0,0.0,0.0,0.0,0,...,-4.379447,0,-7.034504,0,1.685034,1.866787,-1.224098,0,0,0
3,2.483511,-4.166932,0,-1.000000,0.000000,0,0.0,0.0,0.0,0,...,-2.126003,0,-7.083308,0,-0.678377,0.334172,-0.642057,0,0,0


In [86]:
Qt

{0: [2.5922455220835943,
  2.678050560830273,
  2.7707767445079576,
  2.483510642076735],
 1: [1.6961221147101735,
  0.1321061120790061,
  -2.677125674339557,
  -4.166932174592382],
 2: [0, 0, 0, 0],
 3: [-0.7642, -0.0006808368906141005, 0.07500887123422971, -1.0],
 4: [0.2864607926697422, 0, 0, 0],
 5: [0, 0, 0, 0],
 6: [-1.0, 0, 0, 0],
 7: [-0.1, 0, 0, 0],
 8: [0.1, 0, 0, 0],
 9: [0, 0, 0, 0],
 10: [2.285619341115687,
  2.448281910960343,
  2.210012404192539,
  1.7720645522063625],
 11: [1.1623360575905248,
  -0.883737795342324,
  -3.0841065567474684,
  -2.5777490331483315],
 12: [0, 0, 0, 0],
 13: [-1.9546185014774957,
  -1.0781010751138784,
  -0.9512626397894991,
  -1.0462329486059312],
 14: [-2.655780955701838,
  -4.11609653745338,
  -2.7839450348313437,
  -2.7595561804858764],
 15: [0, 0, 0, 0],
 16: [0, 0, 0, 0],
 17: [0, 0, 0, 0],
 18: [-0.1, 0, 0, 0],
 19: [0, 0, 0, 0],
 20: [1.9807369467092588,
  1.9663533441409342,
  1.9708163882298877,
  1.6120962481312504],
 21: [0.7781480

In [10]:
    def reset():
        self.state = 0 
        return self.state

In [11]:
def actionspace():
    return random.choice([0, 1, 2, 3])

In [20]:
def act(state):
    if random.uniform(0, 1) < eps:
        return env.action_space.sample()
    else:
        return max(range(nact), key=lambda a: Qt[state][a])

In [89]:
def ward(state, action, nextstate, reward, done):
    if done and nextstate == goalstate:  
        return 100
    elif done: 
        return -10
    else:
        current_row, current_col = state // 4, state % 4
        next_row, next_col = nextstate // 4, nextstate % 4
        goal_row, goal_col = goalstate // 4, goalstate % 4
        current_distance = abs(current_row - goal_row) + abs(current_col - goal_col)
        next_distance = abs(next_row - goal_row) + abs(next_col - goal_col)
        if next_distance < current_distance: 
            return 1
        elif next_distance > current_distance:  
            return -1
        else:  
            return 0

In [90]:
def qlearn():
    for episode in range(numep):
        state = env.reset()[0] 
        done = False
        tr = 0  
        while not done:
            action = act(state) 
            nextstate, reward, done, _, _ = env.step(action)
            reward=ward(state, action, nextstate, reward, done)
            bestnext = max(range(nact), key=lambda a: Qt[nextstate][a])  
            Qt[state][action] += alpha * (reward + gam * Qt[nextstate][bestnext] - Qt[state][action])  
            state = nextstate
            tr += reward 

In [93]:
qlearn()

KeyboardInterrupt: 

### 3. Testing Your Model
In the cell seen below, write the code you need to test your Q-Learning model for **1000 episodes**. It is important to test your model for 1000 episodes so that we are all able to compare our results.

*Note, level 5 testing uses both a success rate and an average steps taken metric to evaluate your model. Level 4 uses one or the other.*

In [ ]:
# Test model here.

### 4. Final Answer
In the first cell below, describe the path your elf takes to get to the gift. *Note, a level 5 answer includes a gif of the path your elf takes in order to reach the gift.*

In the second cell seen below, describe how well your Q-Learning model performed. Make sure that you explicitly name the **learning rate**, **the discount factor**, and the **reward system** that you used when training your final model. *Note, a level 5 description describes the model's performance using two types of quantitative evidence.*

![My Gif](FrozenLake2025-02-1110-53-26-ezgif.com-video-to-gif-converter.gif)


#### its going backand forth trying to find a closer spot to the gift 

#### pretty good reached the goal often